In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/DataStructures.ji for module DataStructures.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/StatsFuns.ji for module StatsFuns.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Distances.ji for module Distances.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/NearestNeighbors.ji for module NearestNeighbors.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Devectorize.ji for module Devectorize.
  likely near /home/ubuntu/.julia/v0.5/Devectorize/src/texpr.jl:293
  likely near /home/ubuntu/.julia/v0.5/Devectorize/src/texpr.jl:293
  likely near /home/ubuntu/.julia/v0.5/NearestNeighbors/src/knn.jl:1
  likely near /home/

In [2]:
const WN_PATH="data/corpora/WordNet-2.1/"
db = DB(WN_PATH)

WordNet.DB

In [3]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [4]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

In [5]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


punctuation_space_tokenize (generic function with 1 method)

In [13]:
function lexically_informed_embeddings(db,ee, word)
    source_wvs = ee.embedding[word]
    target_synsets = Base.flatten((haskey(db.lemmas[pos], word) ? synsets(db, db[pos, word]) : WordNet.Synset[] 
        for pos in keys(WordNet.SYNSET_TYPES))) 

    if length(source_wvs) == 1
        #Use MCWS
        lemma = db['n', lemma_word]
        target_synsets = synsets(db, lemma);
        score, index = findmax((get(ss.word_counts,lemma_word,0)+0.1sum(values(ss.word_counts))) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
    end
            
    synsets4vectors = Dict{eltype(source_wvs), Synset}()
    for synset::Synset in target_synsets
            context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
            wv = synthesize_embedding(ee,context, word)
            synsets4vectors[wv] = synset
    end
    synsets4vectors
end

lexically_informed_embeddings (generic function with 1 method)

In [17]:
@time lexically_informed_embeddings(db,ee, "federal")

  0.018722 seconds (102.72 k allocations: 1.725 MB)


Dict{Array{Float32,1},WordNet.Synset} with 6 entries:
  Float32[-0.0537537,0.0682679,-0.0259331,0.157535,-0.272935,0.186007,-0.0694081,-0.0785202,0.199033,-0.126381,-0.0847547,0.128801,-0.183493,-0.0422632,0.0892356,-0.22934,-0.0623469,-0.00149297,0.056994,0.0964768,0.0161398,0.0781163,0.211676,-0.0763543,0.144749,0.216146,0.0675393,0.088937,-0.118281,-0.356966,0.0163716,-0.0818849,0.0128478,-0.0469062,-0.0810498,0.203175,0.0173182,0.0536753,0.0529723,-0.0489318,0.0774052,-0.0222965,-0.0963653,0.0107006,0.0746002,-0.082665,-0.199656,-0.186337,-0.0727619,-0.0921054,0.103729,0.0355965,0.0991122,0.114297,0.0437884,0.0641925,-0.155373,-0.191694,0.106501,0.0299754,0.202594,0.0104121,0.00473919,-0.049686,0.203001,0.149391,-0.0946662,-0.00488328,0.0074259,-0.189963,0.206126,0.0392408,-0.0732178,-0.0386577,0.0215429,0.181814,0.00604329,0.0138382,-0.0173619,-0.0353349,-0.0625751,-0.151501,0.00638058,0.139065,0.130912,0.0440749,0.156332,0.108061,0.0793152,0.0144482,0.140982,0.0163569,-0.061246,0

In [22]:
function load_sense_key_map(wn_basedir)
    path=joinpath(wn_basedir, "dict","index.sense")
    sense_key_map = Dict{Tuple{Int64,String},String}()
    for line in eachline(path)
        full_sense_key, offset_str, sense_num_str, tagcount_str = split(line) 
        lemma_name = first(split(full_sense_key,'%'))
        sense_offset = parse(Int64, offset_str)
        index = (sense_offset,lemma_name)
        @assert(!haskey(sense_key_map, index))
        sense_key_map[index] = full_sense_key
    end
    sense_key_map
end


function sense_key(lem::Lemma, synset::Synset,sense_key_map)
    sense_key_map[(s1.offset,lem.word)]
end
function sense_keys(lem::Lemma,sense_key_map)
    [sense_key_map[(ss_offset,lem.word)] for ss_offset in lem.synset_offsets ]
end

sense_keys (generic function with 1 method)

In [24]:
sense_key_map = load_sense_key_map(WN_PATH);

In [25]:
sense_key(l1,s1, sense_key_map)
sense_keys(db['a',"six"],sense_key_map)

LoadError: LoadError: UndefVarError: l1 not defined
while loading In[25], in expression starting on line 1

In [28]:
sense_key_map[2134693,"discover"]

"discover%2:39:03::"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [30]:
using LightXML

INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/LightXML.ji for module LightXML.
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in @windows(::Any, ::Any) at ./deprecated.jl:512
 in include_from_node1(::String) at ./loading.jl:426
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in process_options(::Base.JLOptions) at ./client.jl:243
 in _start() at ./client.jl:322
while loading /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl, in expression starting on line 19


In [31]:
immutable wsd_challenge
    id
    word
    lemma
    pos
    context
end

function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = attribute(lemma_node,"pos")
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(wsd_challenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges() |> collect

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:0 [inlined]
 in content(::LightXML.XMLNode) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:172
 in (::##9#11{LightXML.XMLElement})() at ./In[31]:18
while loading In[31], in expression starting on line 32
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:0 [inlined]
 in #attribute#1(::Bool, ::Function, ::LightXML.XMLElement, ::String) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:227
 in (::##9#11{LightXML.XMLElement})() at ./In[31]:22
while loading In[31], in expression starting on line 32


2269-element Array{wsd_challenge,1}:
 wsd_challenge("d001.s001.t001","editorial","editorial","n",SubString{String}["your","oct","6","the","ill","homeless","referred","to","research","by","us","and","six","of","our","colleagues","that","was","reported","in","the","sept","8","issue","of","the","journal","of","the","american","medical","association"])                                                                                                                                                                                                                                                                                                                                                                       
 wsd_challenge("d001.s001.t002","ill","ill","a",SubString{String}["your","oct","6","editorial","the","homeless","referred","to","research","by","us","and","six","of","our","colleagues","that","was","reported","in","the","sept","8","issue","of","the","journal","of","the","american","medical

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [65]:
immutable WsdSolution
    id
    lemma
    pos
    solutions
end

function load_solutions(key_file="data/corpora/wsd/semeval2007_t7/key/dataset21.test.key")
    map(eachline(key_file)) do line
        line_data, comment = split(line,"!!")
        fields = split(line_data)
        doc_id = fields[1]
        instance_id = fields[2]
        solutions = fields[3:end]
        
        lemma,pos = match(r"lemma=(.*)#(.)", comment).captures
        WsdSolution(instance_id, lemma, pos, solutions)
    end
end

solutions = load_solutions()

2269-element Array{WsdSolution,1}:
 WsdSolution("d001.s001.t001","editorial","n",SubString{String}["editorial%1:10:00::"])                                                                                                                                                                                                                                                                                            
 WsdSolution("d001.s001.t002","Ill","a",SubString{String}["ill%3:00:01::"])                                                                                                                                                                                                                                                                                                        
 WsdSolution("d001.s001.t003","Homeless","n",SubString{String}["homeless%1:14:00::","homeless%1:18:00::"])                                                                                                                   

In [72]:
using Training

In [75]:
type LazyLexicallyInformedWordSenseEmbedding <: WordSenseEmbedding
end

LoadError: LoadError: syntax: incomplete: premature end of input
while loading In[75], in expression starting on line 1

In [74]:
Training.WSD

# 1 method for generic function "WSD":
WSD{S<:String}(embed::WordEmbeddings.WordSenseEmbedding, word::String, context::AbstractArray{S,1}; skip_oov)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl
